# LangChain Expression Language (LECL)

LangChain Expression Language (LEL) is a **declarative scripting language** used in the LangChain framework. It allows developers to define **sequences of operations** for processing and managing language models, data, and interactions with other systems.


Ref link:



**Introduction to LCEL (LangChain Expression Language)**

https://youtu.be/POVaB5AfzC4



**LangChain Expression Language (LCEL) | Langchain Tutorial | Code**

https://youtu.be/NQWfvhw7OcI


**LangChain Expression language(LCEL) for Chaining the Components | All Runnables | Async & Streaming**

https://youtu.be/8aUYzb1aYDU

In [110]:
!pip install -q langchain_google_genai
!pip install -q langchain_community
!pip install -q langchain
!pip install -q langchain_huggingface
!pip install -q chromadb
!pip install -q tiktoken
!pip install -q bs4
!pip install -q python-dotenv

## Setup the Environment

In [111]:
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
import bs4


In [112]:
## Model
import os

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
google_llm = ChatGoogleGenerativeAI(model='gemini-1.0-pro')

## Huggingface Embeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [113]:
## Load Documents
loader = WebBaseLoader(
    web_paths=[
        "https://lilianweng.github.io/posts/2023-06-23-agent/"
    ],
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title","post-header")
        )
    )
)
document = loader.load()

In [114]:
## Text Splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500,
    chunk_overlap=50
)
splits = text_splitter.split_documents(document)
print("No of chunks : ", len(splits))
splits[0]

No of chunks :  31


Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refin

In [115]:
## VectoreStore
vectorestore = Chroma.from_documents(
    documents=splits,
    embedding=hf_embeddings
)
vectorestore

In [116]:
## Retriever
retriever = vectorestore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7c7968f87cd0>, search_kwargs={})

# LECL

### Topic Covered

1. **Invoke** - Invoke chain with input

2. **Batch** - Invoke chain with list of inputs

3. **Stream** - Streaming the response

4. **RunnablePassthrough** - Feed input to chain

5. **RunnableParallel** - Parallel execution

6. **RunnableLambda** - Use user_defined_functions

7. **ainvoke** - asynchronously invokes chain with input

8. **abatch** - asynchronously invokes chain with list of inputs

In [130]:
template = """Provide me the 5 summary line of the following topic from the context.
{context}

topic : {topic}
"""
prompt = ChatPromptTemplate.from_template(template)


In [118]:
chain = prompt | google_llm

chain

ChatPromptTemplate(input_variables=['context', 'topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'topic'], input_types={}, partial_variables={}, template='Provide me the 5 summary line of the following topic from the context.\n{context}\n\ntopic : {topic}\n'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-1.0-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c796914d060>, default_metadata=())

Invoke method

In [119]:
## Simple Invoke method
chain = prompt | google_llm

context_ = retriever.get_relevant_documents ("Task Decomposition")
chain.invoke({'context' : context_,'topic' : 'Task Decomposition'})

AIMessage(content='1. Task decomposition is crucial for planning complex tasks in autonomous agent systems.\n2. Chain of thought (CoT) and Tree of Thoughts (ToT) are prompting techniques that facilitate task decomposition by breaking down problems into smaller steps.\n3. Task decomposition can be performed by LLM with simple prompts, task-specific instructions, or human inputs.\n4. LLM+P (LLM and Planner) leverages an external classical planner to handle long-horizon planning through PDDL (Planning Domain Definition Language).\n5. Task decomposition enables agents to generate a plan of action, identify subgoals, and efficiently execute complex tasks.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'cate

In [120]:
## StrOutputParser
chain = prompt | google_llm | StrOutputParser()

context_ = retriever.get_relevant_documents ("Task Decomposition")
chain.invoke({'context' : context_,'topic' : 'Task Decomposition'})

'1. Task Decomposition is a key component of LLM-powered autonomous agents, enabling them to break down complex tasks into manageable steps.\n2. Chain of Thought (CoT) and Tree of Thoughts (ToT) are prompting techniques that guide LLMs to decompose tasks step-by-step, providing insights into their reasoning process.\n3. Task decomposition can be performed by LLMs with simple prompting, task-specific instructions, or human inputs.\n4. LLM+P is an alternative approach that utilizes an external classical planner to generate a plan in the Planning Domain Definition Language (PDDL), which is then translated by the LLM.\n5. The Planning stage in an LLM-powered agent system involves task parsing, model selection, and task execution, with LLMs playing a central role in coordinating these processes.'

In [121]:
## StrOutputParser + Split the line function
chain = prompt | google_llm | StrOutputParser() | (lambda x : x.split('\n'))

context_ = retriever.get_relevant_documents ("Task Decomposition")
chain.invoke({'context' : context_,'topic' : 'Task Decomposition'})

['1. Task decomposition is essential for complex tasks, allowing an agent to plan ahead by breaking them into smaller steps.',
 "2. Chain of Thought (CoT) and Tree of Thoughts extend decomposition by providing a step-by-step interpretation of the model's reasoning process.",
 '3. Task decomposition can be performed by the LLM itself, through external instructions, or with human input.',
 '4. LLM+P involves outsourcing planning to an external planner using the Planning Domain Definition Language (PDDL).',
 '5. Self-reflection enhances planning by allowing the agent to evaluate and refine its task decomposition strategies.']

Batch

In [122]:
## StrOutputParser + Split the line function
chain = prompt | google_llm | StrOutputParser() | (lambda x: x.split('\n'))

context_sensory_memory = retriever.get_relevant_documents ("Sensory Memory")
context_short_term_memory = retriever.get_relevant_documents ("Short-Term Memory")
context_long_term_memory = retriever.get_relevant_documents ("Long-Term Memory")
context_mips = retriever.get_relevant_documents ("Maximum Inner Product Search (MIPS)")

chain.batch([
    {'context' : context_sensory_memory,'topic' : 'Sensory Memory'},
    {'context' : context_short_term_memory,'topic' : 'Short-Term Memory'},
    {'context' : context_long_term_memory,'topic' : 'Long-Term Memory'},
    {'context' : context_mips,'topic' : 'Maximum Inner Product Search (MIPS)'},
    ])

[['1. Sensory memory is the initial stage of memory that stores impressions of sensory information for a few seconds.',
  '2. It includes iconic memory (visual), echoic memory (auditory), and haptic memory (touch).',
  '3. Sensory memory is analogous to learning embedding representations for raw inputs in AI models.',
  '4. The short-term memory capacity is limited and lasts for approximately 20-30 seconds.',
  '5. Short-term memory is comparable to in-context learning in AI models, which is restricted by the finite context window length.'],
 ['1. Short-term memory (STM) stores information currently in use and is essential for cognitive tasks.',
  '2. STM has a limited capacity of approximately 7 items and lasts for 20-30 seconds.',
  '3. In AI, sensory memory is analogous to embedding representations of raw inputs, while STM corresponds to in-context learning within a finite context window.',
  '4. Long-term memory is represented as an external vector store accessible via fast retriev

Stream - Streaming the output

In [123]:
template = """Provide me the 500 words summary of the following topic from the context.
{context}

topic : {topic}
"""
prompt = ChatPromptTemplate.from_template(template)

chain = prompt | google_llm | StrOutputParser()

context_ = retriever.get_relevant_documents ("Task Decomposition")


for s in chain.stream({'context' : context_,'topic' : 'Task Decomposition'}):
  print(s, end='', flush=True)

**Task Decomposition for Autonomous Agents**

Task decomposition is a crucial aspect of planning for autonomous agents, as it enables them to break down complex tasks into smaller, manageable steps. This process involves identifying the subtasks that need to be completed and the order in which they should be executed.

**Chain of Thought (CoT)**

CoT is a widely adopted prompting technique that enhances model performance on complex tasks. It instructs the model to "think step by step," decomposing hard tasks into smaller steps by utilizing more test-time computation. CoT transforms large tasks into multiple manageable ones, offering insights into the model's thought process.

**Tree of Thoughts (ToT)**

ToT extends CoT by exploring multiple reasoning possibilities at each step. It decomposes the problem into thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS or DFS, with each state evaluated by a classifier or majority vote.

## Runnable Protocol

In [124]:
template = """Provide me the 100 words summary of the following topic from the context.
{context}

topic : {topic}
"""
prompt = ChatPromptTemplate.from_template(template)

RunnablePassthrough

In [125]:
chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | google_llm
    | StrOutputParser()
)

chain.invoke('Maximum Inner Product Search (MIPS)')

'Maximum Inner Product Search (MIPS) is a technique that enables fast retrieval of relevant information from an external vector store. ANN algorithms, such as approximate nearest neighbors, are commonly used for MIPS to efficiently return the top k nearest neighbors based on inner product similarity. This allows agents to access a vast amount of information beyond their immediate context, extending their capabilities and alleviating the limitations of finite attention span.'

In [126]:
chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | google_llm
    | StrOutputParser()
)

chain.batch(['Task Decomposition', 'Types of Memory','Maximum Inner Product Search (MIPS)'])

['Task decomposition is a technique used in planning for complex tasks. It involves breaking down the task into smaller, manageable steps. This can be done using various methods, such as chain of thought (CoT), tree of thoughts, LLM+P, or simple prompting. CoT instructs the model to "think step by step" to utilize test-time computation to decompose hard tasks into smaller steps. Tree of thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure. LLM+P relies on an external classical planner to do long-horizon planning, utilizing the Planning Domain Definition Language (PDDL). Simple prompting involves using LLM to decompose the task with prompts like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?". Task decomposition helps agents plan ahead and understand the steps involved in completing a task.',
 "Memory involves acquiring, storing, and retrieving information. Human memory is categorized into:\n\n* **Sensory Memory

RunnableLambda

In [127]:
def string_upper(input: str) -> str:
    return input.upper()

chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | google_llm
    | StrOutputParser()
    | RunnableLambda(string_upper)
)

chain.batch(['Task Decomposition', 'Types of Memory','Maximum Inner Product Search (MIPS)'])

['TASK DECOMPOSITION IS A CRUCIAL ASPECT OF PLANNING FOR AUTONOMOUS AGENTS, AS IT ALLOWS COMPLEX TASKS TO BE BROKEN DOWN INTO SMALLER, MORE MANAGEABLE STEPS. CHAIN OF THOUGHT (COT) IS A STANDARD PROMPTING TECHNIQUE THAT ENHANCES MODEL PERFORMANCE ON COMPLEX TASKS BY INSTRUCTING THE MODEL TO "THINK STEP BY STEP." TREE OF THOUGHTS EXTENDS COT BY EXPLORING MULTIPLE REASONING POSSIBILITIES AT EACH STEP. TASK DECOMPOSITION CAN BE DONE BY LLM WITH SIMPLE PROMPTING, TASK-SPECIFIC INSTRUCTIONS, OR HUMAN INPUTS. A DISTINCT APPROACH, LLM+P, INVOLVES RELYING ON AN EXTERNAL CLASSICAL PLANNER TO DO LONG-HORIZON PLANNING. SELF-REFLECTION ALLOWS AGENTS TO MONITOR THEIR OWN PERFORMANCE AND IDENTIFY AREAS FOR IMPROVEMENT.',
 'MEMORY, THE ABILITY TO STORE AND RETRIEVE INFORMATION, IS ESSENTIAL FOR COGNITIVE FUNCTION. HUMAN MEMORY CAN BE CLASSIFIED INTO THREE MAIN TYPES:\n\n1. **SENSORY MEMORY:** RETAINS SENSORY INFORMATION FOR A FEW SECONDS (E.G., ICONIC MEMORY FOR VISUAL STIMULI, ECHOIC MEMORY FOR AUDI

Without RunnableLambda

In [131]:
def string_upper(input: str) -> str:
    return input.upper()

chain = (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt
    | google_llm
    | StrOutputParser()
    | string_upper
)

chain.batch(['Task Decomposition', 'Types of Memory','Maximum Inner Product Search (MIPS)'])

['1. TASK DECOMPOSITION IS CRUCIAL FOR LLM-POWERED AUTONOMOUS AGENTS TO PLAN AND EXECUTE COMPLEX TASKS.\n2. TECHNIQUES FOR TASK DECOMPOSITION INCLUDE CHAIN OF THOUGHT (COT), TREE OF THOUGHTS, AND EXTERNAL CLASSICAL PLANNERS.\n3. COT AND TREE OF THOUGHTS GUIDE THE LLM TO BREAK DOWN TASKS INTO SMALLER, MANAGEABLE STEPS.\n4. EXTERNAL CLASSICAL PLANNERS, LIKE LLM+P, PROVIDE AN ALTERNATIVE APPROACH BY TRANSLATING THE PROBLEM INTO A FORMAT SUITABLE FOR PLANNING TOOLS.\n5. TASK DECOMPOSITION CAN BE ACHIEVED THROUGH LLM PROMPTING, TASK-SPECIFIC INSTRUCTIONS, OR HUMAN INPUT.',
 '1. MEMORY INVOLVES ACQUIRING, STORING, RETAINING, AND RETRIEVING INFORMATION.\n2. SENSORY MEMORY RETAINS SENSORY IMPRESSIONS FOR A FEW SECONDS (ICONIC, ECHOIC, HAPTIC).\n3. SHORT-TERM MEMORY HOLDS INFORMATION FOR 20-30 SECONDS, WITH A LIMITED CAPACITY.\n4. LONG-TERM MEMORY STORES INFORMATION FOR EXTENDED PERIODS, WITH UNLIMITED CAPACITY.\n5. LONG-TERM MEMORY INCLUDES EXPLICIT (FACTS, EVENTS) AND IMPLICIT (SKILLS, ROUTIN

RunnableParallel

In [132]:
template1 = """Provide me the 100 words summary of the following topic from the context.
{context}

topic : {topic}
"""
prompt1 = ChatPromptTemplate.from_template(template1)

chain1= (
    {"context": retriever, "topic": RunnablePassthrough()}
    | prompt1
    | google_llm
    | StrOutputParser()
)



In [133]:
template2 = """Provide me the 5 advantage of the following
topic based on your understanding.

Output should be in Json format
topic : {topic}
"""
prompt2 = ChatPromptTemplate.from_template(template2)

chain2= (
    prompt2
    | google_llm
    | JsonOutputParser()
)


In [134]:
template3 = """What is the advantage of {topic}"
"""
prompt3 = ChatPromptTemplate.from_template(template3)

chain3= (
    prompt3
    | google_llm
    | StrOutputParser()
    | RunnableLambda(string_upper)
)


In [135]:
combined = RunnableParallel(from_context=chain1, from_llm_knowledge=chain2, advantage = chain3)

combined.invoke('Decomposition')

{'from_context': '**Planning in Autonomous Agents using LLMs**\n\nLarge Language Models (LLMs) are increasingly used as the core controllers in autonomous agents. Planning is a key component of these agents, enabling them to break down complex tasks into manageable subgoals.\n\n**Subgoal and Decomposition**\n\nLLMs can use techniques like Chain of Thought (CoT) and Tree of Thoughts (ToT) to decompose tasks into smaller steps. This allows them to handle complex tasks efficiently.\n\n**Reflection and Refinement**\n\nAgents can also engage in self-criticism and self-reflection to refine their plans. They can learn from past mistakes and improve the quality of their future actions.\n\n**Task Decomposition Approaches**\n\nLLMs can decompose tasks using various methods:\n\n* Simple prompting (e.g., "Steps for XYZ")\n* Task-specific instructions (e.g., "Write a story outline")\n* External classical planners (e.g., using PDDL)\n\nBy incorporating planning into autonomous agents, LLMs enable th

## Asynchronous

Asynchronous in LCEL (LangChain Expression Language) refers to the ability to run chains and components concurrently, without blocking other operations. This is especially useful in LangChain when you're working with multiple LLM (Language Model) calls, retrievers, or external API calls that might take time to complete.

In [136]:
import asyncio
import nest_asyncio

nest_asyncio.apply()

ainvoke

In [137]:
## Invoke
template = """Provide me the 5 summary line of the following topic from the context.
{context}

topic : {topic}
"""
prompt = ChatPromptTemplate.from_template(template)

## StrOutputParser + Split the line function
chain = prompt | google_llm | StrOutputParser() | (lambda x : x.split('\n'))

context_ = retriever.get_relevant_documents ("Task Decomposition")

# asyncio.run(chain.ainvoke({'context' : context_,'topic' : 'Task Decomposition'}))
await chain.ainvoke({'context' : context_,'topic' : 'Task Decomposition'})

['1. Task decomposition involves breaking down complex tasks into smaller, manageable steps.',
 '2. Chain of Thought (CoT) and Tree of Thoughts (ToT) are prompting techniques that guide models in decomposing tasks.',
 '3. Task decomposition can be performed by the LLM itself, using task-specific instructions, or with human input.',
 '4. LLM+P approach outsources planning to an external classical planner, utilizing Planning Domain Definition Language (PDDL).',
 '5. Task decomposition enables LLM-powered agents to plan and execute complex tasks more effectively.']

abatch

In [138]:
## StrOutputParser + Split the line function
chain = prompt | google_llm | StrOutputParser() | (lambda x: x.split('\n'))

context_sensory_memory = retriever.get_relevant_documents ("Sensory Memory")
context_short_term_memory = retriever.get_relevant_documents ("Short-Term Memory")
context_long_term_memory = retriever.get_relevant_documents ("Long-Term Memory")
context_mips = retriever.get_relevant_documents ("Maximum Inner Product Search (MIPS)")

await chain.abatch([
    {'context' : context_sensory_memory,'topic' : 'Sensory Memory'},
    {'context' : context_short_term_memory,'topic' : 'Short-Term Memory'},
    {'context' : context_long_term_memory,'topic' : 'Long-Term Memory'},
    {'context' : context_mips,'topic' : 'Maximum Inner Product Search (MIPS)'},
    ])

[['1. Sensory memory is the initial stage of memory, retaining sensory information for a few seconds.',
  '2. Subcategories of sensory memory include iconic (visual), echoic (auditory), and haptic (touch) memory.',
  '3. Sensory memory is comparable to learning embedding representations for raw inputs in machine learning.',
  '4. Sensory memory provides the foundation for short-term and long-term memory formation.',
  '5. The duration of sensory memory is typically limited to a few seconds or less.'],
 ['1. Human memory can be categorized into sensory, short-term, and long-term memory.',
  '2. Sensory memory retains sensory information for a few seconds, while short-term memory stores information needed for immediate cognitive tasks.',
  '3. Long-term memory has unlimited storage capacity, and can store information for prolonged periods, including declarative (facts and events) and procedural (skills and routines) memories.',
  '4. External memory, accessible via fast retrieval, can al